# Visualização de dados Geográficos

Podemos criar mapas com diferentes fontes de mapas. Isso é possível através do [TileSource](https://bokeh.pydata.org/en/latest/docs/user_guide/geo.html#tile-providers), em especial WMTSTileSource, que permite que os dados sejam sobrepostos em qualquer servidor, incluindo Google Maps, OpenStreatMap, Stamen, MapQuest, ou algum servidor próprio. 

Esse notebook apresenta um exemplo da utilização de uma mapa através do OpenStreatMap.

## Carregando o conjunto de dados

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('tweets_10min.csv', sep=';')

In [ ]:
df.head(1)

In [ ]:
df_aux = df[['created_at', 'lat', 'long','location' ,'screen_name', 'text']].copy()

In [ ]:
df_aux = df_aux.rename(columns={'created_at':'date', 'screen_name':'user', 'long':'lon'})

In [ ]:
df_aux.head(3)

## Criando o Mapa

Primeiro iremos utilizar o padrão mais comum na web, o WTMS (Web Map Tile Service). A ideia principal é que após criado o mapa é possível determinar o nível de zoom e as informações relacionadas a uma determinada região irá aparecer.

WTMS utiliza o formato de Web Mercator, que mede a distâncias de Greenwich, Inglaterra.

Primeiro vamos criar um plot com o mapa mundi.

In [ ]:
from bokeh.io import output_notebook, show

In [ ]:
output_notebook()

In [ ]:
from bokeh.plotting import figure
from bokeh.tile_providers import CARTODBPOSITRON

In [ ]:
bound=20000000 # metros para ter a visão do mundo todo.

In [ ]:
plot = figure(x_range=(-bound,bound), y_range=(-bound,bound))
plot.axis.visible = False

In [ ]:
plot.title.text = "Mostrando a região de {} tweets".format(len(df_aux.index))
plot.title.text_font_size="20pt"

In [ ]:
plot.add_tile(CARTODBPOSITRON)

Outros tipos de mapa pode ser visualizado em https://bokeh.pydata.org/en/latest/docs/reference/tile_providers.html

In [ ]:
show(plot)

Ótimo já temos o nosso mapa, agora precisamos adicionar os pontos que queremos. É possível adicionar qualquer coisa utilizando o Bokeh, porém é necessário obter as coordenadas no formato de Web Mercator.

Nesse caso temos que aplicar a seguinte função no campo de latitude e longitude para converter de decimal.

Essa função foi retirada de: https://goo.gl/sWRnwT

In [ ]:
import numpy as np
def converter_web_mercator(df, lon="lon", lat="lat"):
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

In [ ]:
df_novo = converter_web_mercator(df_aux)

In [ ]:
df_novo.tail()

In [ ]:
plot.circle('x', 'y', 
            fill_color="blue", 
            line_color=None, 
            size=10,
            source=df_novo)

In [ ]:
show(plot)

### Adicionando as informações complementares.

Para isso iremos utilizar a ferramenta chamada HoverTool, que irá gerar uma "janela de dicas" com as informações de cada linha que contém o rótulo (user, location, etc). Em nosso caso, é o nome da coluna do DataFrame.

In [ ]:
from bokeh.models import HoverTool

In [ ]:
plot.add_tools(HoverTool(tooltips=[("user", "@user"),("local", "@location")]))

In [ ]:
show(plot)